In [1]:
using LinearAlgebra

In [2]:
function lagrange_basis(geom="triangular", order=1)
    if (geom == "triangular" && order == 1)
        # basis functions
        ϕ₁(x, y) = 1 - x - y
        ϕ₂(x, y) = x
        ϕ₃(x, y) = y
        ϕ = (ϕ₁, ϕ₂, ϕ₃)
        
        # gradients
        ∇ϕ₁(x, y) = [-1, -1]
        ∇ϕ₂(x, y) = [1, 0]
        ∇ϕ₃(x, y) = [0, 1]
        ∇ϕ = (∇ϕ₁, ∇ϕ₂, ∇ϕ₃)
        
        # quadrature nodes and weights
        x₁ = [0.5, 0]
        x₂ = [0, 0.5]
        x₃ = [0.5, 0.5]        
        nodes = (x₁, x₂, x₃)
        weights = (1/6, 1/6, 1/6)
    end
    
    return ϕ, ∇ϕ, nodes, weights
end

lagrange_basis (generic function with 3 methods)

In [3]:
function quadrature(fun, nodes, weights)
    rv = 0
    for (i, n) in enumerate(nodes)
        rv += fun(n[1], n[2]) * weights[i]
    end
    return rv
end

quadrature (generic function with 1 method)

In [4]:
function element_affine_map(x; v₁=[1 1], v₂=[1 0], v₃=[0 1])
    return (v₂ - v₁) * x[1] + (v₃ - v₁) * x[2] + v₁
end

element_affine_map (generic function with 1 method)

In [8]:
_, ∇ϕ, nodes, weights = lagrange_basis()
f(x, y) = x^2 + x*y + y^2

f (generic function with 1 method)

In [6]:
quadrature(f, element_affine_map.(nodes), weights) + quadrature(f, nodes, weights)

0.9166666666666665

In [7]:
using SymPy
x, y = Sym("x, y")
integrate(x^2 + x*y + y^2, (x, 0, 1), (y, 0, 1))

11
──
12

In [26]:
function get_local_matrix(∇ϕ)
    n = length(∇ϕ)
    rv = zeros(n, n)
    for i ∈ 1:n
        for j ∈ 1:n
            phi(x, y) = dot(∇ϕ[i](x, y), ∇ϕ[j](x, y))
            rv[i, j] = quadrature(phi, nodes, weights)
        end
    end
    return rv
end

get_local_matrix (generic function with 1 method)

In [27]:
get_local_matrix(∇ϕ)

3×3 Matrix{Float64}:
  1.0  -0.5  -0.5
 -0.5   0.5   0.0
 -0.5   0.0   0.5